# Mecklenburg Grocery Stores

#### Author : Sumit Chhabra

In [1]:
#install folium in IBM Watson Studio
#skip this step if already installed
!conda install -c conda-forge folium=0.5.0 --yes

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  48.11 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  35.29 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  37.40 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  48.40 MB/s


# Convert HTML to pandas dataframe

Combine all steps from previous assignment 

In [2]:
#Read from wikipedia
import requests
from bs4 import BeautifulSoup
import pandas as pd


res = requests.get("https://www.zip-codes.com/county/nc-mecklenburg.asp")
soup = BeautifulSoup(res.content, "lxml" )
#soup

#extract table from html
table = soup.find_all('table')[2]
#table

df = pd.read_html(str(table))[0]
print("Old:", df.shape)
df.head()

Old: (84, 6)


,0,1,2,3,4,5
0,ZIP Code,Classification,City,Population,Timezone,Area Code(s)
1,ZIP Code 28031,General,Cornelius,24390,Eastern,704/980
2,ZIP Code 28035,General,Davidson,0,Eastern,704/980
3,ZIP Code 28036,General,Davidson,14654,Eastern,704/980
4,ZIP Code 28070,P.O. Box,Huntersville,0,Eastern,704


In [3]:
import re

#convert first row to columns
headers = df.iloc[0]
new_df  = pd.DataFrame(df.values[1:], columns=headers)
new_df.drop(['Classification', 'Timezone','Area Code(s)'],inplace=True,axis=1)
new_df['ZIP Code'] = new_df['ZIP Code'].apply(lambda x: re.sub('ZIP Code','',x))
print(new_df.shape)
new_df.head()

(83, 3)


,ZIP Code,City,Population
0,28031,Cornelius,24390
1,28035,Davidson,0
2,28036,Davidson,14654
3,28070,Huntersville,0
4,28078,Huntersville,52133


In [4]:
#data cleanup. remove zip codes with 0 population
new_df.drop(new_df.loc[new_df['Population']== '0'].index, inplace=True)
print(new_df.shape)
new_df.head()

(30, 3)


,ZIP Code,City,Population
0,28031,Cornelius,24390
2,28036,Davidson,14654
4,28078,Huntersville,52133
5,28105,Matthews,39586
9,28134,Pineville,9453


In [5]:
#focus on Charlotte city only
new_df = new_df[new_df['City'].str.contains("Charlotte")]
print(new_df.shape)
new_df.head()

(25, 3)


,ZIP Code,City,Population
11,28202,Charlotte,11195
12,28203,Charlotte,11315
13,28204,Charlotte,4796
14,28205,Charlotte,43931
15,28206,Charlotte,11898


In [6]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

def getLocation(address):
    geolocator = Nominatim(user_agent="clt_explorer")
    location = geolocator.geocode(address)
    return location

In [7]:

#takes upto 30 secs to run
for index, row in new_df.iterrows():
    location = getLocation(row['ZIP Code']) 
    new_df.at[index, 'Longitude'] = location.longitude
    new_df.at[index, 'Latitude'] = location.latitude

new_df = new_df.reset_index(drop=True)
new_df

,ZIP Code,City,Population,Longitude,Latitude
0,28202,Charlotte,11195,-80.839331,35.234558
1,28203,Charlotte,11315,-80.875865,35.202613
2,28204,Charlotte,4796,-80.827106,35.217039
3,28205,Charlotte,43931,-80.798422,35.207553
4,28206,Charlotte,11898,-80.828205,35.250124
5,28207,Charlotte,9280,8.866428,53.066673
6,28208,Charlotte,34167,-80.955640,35.182624
7,28209,Charlotte,20317,-80.875952,35.173773
8,28210,Charlotte,42263,-80.847199,35.126205
9,28211,Charlotte,28523,-80.787204,35.167513


In [8]:
#convert to float
new_df.Population = new_df.Population.astype(float)    
new_df.head()

new_df['ZIP Code'] = pd.to_numeric(new_df['ZIP Code'])
#new_df.head()
new_df['ZIP Code'] = new_df['ZIP Code'].apply(str)
print(new_df['ZIP Code'])

0     28202
1     28203
2     28204
3     28205
4     28206
5     28207
6     28208
7     28209
8     28210
9     28211
10    28212
11    28213
12    28214
13    28215
14    28216
15    28217
16    28226
17    28227
18    28262
19    28269
20    28270
21    28273
22    28277
23    28278
24    28280
Name: ZIP Code, dtype: object


#### Create a map of Charlotte city with zip codes superimposed on top.

In [9]:
import folium

location = getLocation("Charlotte, NC")

# create map of Toronto using latitude and longitude values
map_clt = folium.Map(location=[location.latitude, location.longitude], zoom_start=10)

# add markers to map
for lat, lng, zipcode in zip(new_df['Latitude'], new_df['Longitude'], new_df['ZIP Code']):
    label = '{}'.format(zipcode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_clt)  
    
map_clt

In [10]:
#clt_geo = r'zipcodes.geojson'
import requests
import numpy as np

url = 'https://raw.github.com/mecklenburg-gis/mecklenburg-gis-opendata/master/data/zipcodes.geojson'
clt_geo = requests.get(url).json()

# create a numpy array of length 6 and has linear spacing from the minium total immigration to the maximum total immigration
threshold_scale = np.linspace(new_df['Population'].min(),
                              new_df['Population'].max(),
                              6, dtype=int)
threshold_scale = threshold_scale.tolist() # change the numpy array to a list
threshold_scale[-1] = threshold_scale[-1] + 1

location = getLocation("Charlotte, NC")

# create map of Charlotte using latitude and longitude values
map_clt = folium.Map(location=[location.latitude, location.longitude], zoom_start=10)

# generate choropleth map 
map_clt.choropleth(
    geo_data=clt_geo,
    data=new_df,
    columns=['ZIP Code','Population'],
    key_on='feature.properties.zipcode',
    threshold_scale=threshold_scale,
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Charlotte Zip Code Boundaries',
    reset=True
)

# display map
map_clt



In [ ]:
#convert to dataframe
df = pd.read_html(str(table))[0]
print("Old:", df.shape)
#convert first row to columns and drop first row
headers = df.iloc[0]
new_df  = pd.DataFrame(df.values[1:], columns=headers)
new_df.rename(columns={'Postcode': 'PostalCode'}, inplace=True)
#print(new_df.head())

#drop Borough with "Not assigned"
new_df = new_df[~new_df['Borough'].isin([NA])]
#new_df.head()
print("New:", new_df.shape)

#Replace Not assigned Neighborhood with Borough name
for index, row in new_df.iterrows():
    if row['Neighbourhood'] == NA:
        print ('Found ', row['Borough'], ' - replace it')
        new_df.at[index, 'Neighbourhood'] = row['Borough']

#backup method to get latitude and longitude
url="http://cocl.us/Geospatial_data"
geodata=pd.read_csv(url)
geodata.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

#merge two dataframes
merged_df = new_df.merge(geodata, how = 'inner', on = ['PostalCode'])
print(merged_df.loc[merged_df['PostalCode'] == 'M5G'])
print(merged_df.loc[merged_df['PostalCode'] == 'M5V'])

#### Use geopy library to get the latitude and longitude values of Toronto City.
In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent toronto_explorer, as shown below.

In [11]:
#install geocoder in IBM Watson Studio
#skip this step if already installed
!conda install -c conda-forge geopy --yes

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  14.94 MB/s
geopy-1.18.1-p 100% |################################| Time: 0:00:00  35.42 MB/s


#### Define Foursquare Credentials and Version

In [12]:
#Note to provide your creds

CLIENT_ID = 'GWAG0Y3R0AQZRZLEYFFD4CZGBUM14TWYE3YQRZUY0XFBOSGB' # your Foursquare ID
CLIENT_SECRET = '4IZY24VFB03REW5SZKLHP1N4YMLSWPQDUH0DY2CBRRZKOYMT' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GWAG0Y3R0AQZRZLEYFFD4CZGBUM14TWYE3YQRZUY0XFBOSGB
CLIENT_SECRET:4IZY24VFB03REW5SZKLHP1N4YMLSWPQDUH0DY2CBRRZKOYMT


In [35]:
# create the API request URL
url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        location.latitude, 
        location.longitude, 
        5000, 
        100,
        '4bf58dd8d48988d118951735')
results = requests.get(url).json()["response"]['venues']
print(len(results))
results


6


[{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_',
     'suffix': '.png'},
    'id': '4bf58dd8d48988d118951735',
    'name': 'Grocery Store',
    'pluralName': 'Grocery Stores',
    'primary': True,
    'shortName': 'Grocery Store'}],
  'hasPerk': False,
  'id': '4bb70e2e6edc76b051c5311c',
  'location': {'cc': 'ES',
   'country': 'España',
   'distance': 1380,
   'formattedAddress': ['España'],
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.59452127342838,
     'lng': -4.128945565100577}],
   'lat': 40.59452127342838,
   'lng': -4.128945565100577},
  'name': 'Autoservicio DIA',
  'referralId': 'v-1553290183'},
 {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_',
     'suffix': '.png'},
    'id': '4bf58dd8d48988d118951735',
    'name': 'Grocery Store',
    'pluralName': 'Grocery Stores',
    'primary': True,
    'shortName': 'Grocery Store'}],
  'hasPerk': False,
  'id': '4e195e89d22

In [41]:
print((v['name'], v['location']['lat'], v['location']['lng']) for v in results)

<generator object <genexpr> at 0x7f7ad63a7a98>


In [81]:
grocery_df=pd.DataFrame(columns=['ZIP Code', 'Grocery Store', 'Latitude', 'Longitude'])

cnt = []
for index, row in new_df.iterrows():
    #print(row['Latitude'])
    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            row['Latitude'], 
            row['Longitude'], 
            3000, 
            100,
            '4bf58dd8d48988d118951735')
        
    results2 = requests.get(url).json()["response"]['venues']

    cnt.append(len(results2))
    
    #print(len(results))
    for v in results2:
        #print(v['name'])
        grocery_df = grocery_df.append({'ZIP Code' : row['ZIP Code'], 
                        'Grocery Store' : v['name'], 
                        'Latitude' : v['location']['lat'],
                        'Longitude' : v['location']['lng']}, ignore_index=True)
        

new_df['Count'] = cnt
print(grocery_df.shape)
grocery_df.head()

(287, 4)


,ZIP Code,Grocery Store,Latitude,Longitude
0,28202,Harris Teeter,35.231231,-80.843956
1,28202,Cash Lady,35.226659,-80.841906
2,28202,Wayne's Super Market,35.247299,-80.829380
3,28202,Beny's Mart,35.249342,-80.828599
4,28202,Paso's Supermarket,35.247404,-80.844066


In [82]:
new_df

,ZIP Code,City,Population,Longitude,Latitude,Count
0,28202,Charlotte,11195.0,-80.839331,35.234558,18
1,28203,Charlotte,11315.0,-80.875865,35.202613,15
2,28204,Charlotte,4796.0,-80.827106,35.217039,19
3,28205,Charlotte,43931.0,-80.798422,35.207553,16
4,28206,Charlotte,11898.0,-80.828205,35.250124,22
5,28207,Charlotte,9280.0,8.866428,53.066673,14
6,28208,Charlotte,34167.0,-80.955640,35.182624,3
7,28209,Charlotte,20317.0,-80.875952,35.173773,17
8,28210,Charlotte,42263.0,-80.847199,35.126205,6
9,28211,Charlotte,28523.0,-80.787204,35.167513,7


In [70]:
import folium
import requests
import numpy as np

url = 'https://raw.github.com/mecklenburg-gis/mecklenburg-gis-opendata/master/data/zipcodes.geojson'
clt_geo = requests.get(url).json()

# create a numpy array of length 6 and has linear spacing from the minium total immigration to the maximum total immigration
threshold_scale = np.linspace(new_df['Population'].min(),
                              new_df['Population'].max(),
                              6, dtype=int)
threshold_scale = threshold_scale.tolist() # change the numpy array to a list
threshold_scale[-1] = threshold_scale[-1] + 1

location = getLocation("Charlotte, NC")

# create map of Charlotte using latitude and longitude values
map_clt = folium.Map(location=[location.latitude, location.longitude], zoom_start=10)

# generate choropleth map 
map_clt.choropleth(
    geo_data=clt_geo,
    data=new_df,
    columns=['ZIP Code','Population'],
    key_on='feature.properties.zipcode',
    threshold_scale=threshold_scale,
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Charlotte Zip Code Boundaries',
    reset=True
)

# add markers to map
for lat, lng, groceryname in zip(grocery_df['Latitude'], grocery_df['Longitude'], grocery_df['Grocery Store']):
    label = '{}'.format(groceryname)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_clt)  
    
map_clt

## 2. Explore Neighborhoods in Toronto

#### Let's create a function to repeat the same process to all the neighborhoods in Manhattan

In [71]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, limit=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit,
            '4bf58dd8d48988d118951735')
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *toronto_venues*.

In [73]:
toronto_venues = getNearbyVenues(names=new_df['ZIP Code'],
                                   latitudes=new_df['Latitude'],
                                   longitudes=new_df['Longitude']
                                  )
print(toronto_venues.shape)
toronto_venues.head()

KeyError: 'groups'

Let's check how many venues were returned for each neighborhood

In [ ]:
toronto_venues.groupby('Neighborhood').count()

Let's find out how many unique categories can be curated from all the returned venues

In [ ]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

## 3. Analyze Each Neighborhood

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
# seems to be a bug and doesnt like Neighborhood
toronto_onehot['Neighbourhood'] = toronto_venues['Neighborhood'] 
toronto_onehot.shape

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
#print("Fixed Columns:", fixed_columns)
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

#### Let's print each neighborhood along with the top 5 most common venues

In [ ]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [ ]:
import numpy as np

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [ ]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = filtered_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!




Finally, let's visualize the resulting clusters


In [ ]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

#### Cluster 1

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1,2] + list(range(5, toronto_merged.shape[1]))]]


#### Cluster 2

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1,2] + list(range(5, toronto_merged.shape[1]))]]


#### Cluster 3

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1,2] + list(range(5, toronto_merged.shape[1]))]]


#### Cluster 4

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1,2] + list(range(5, toronto_merged.shape[1]))]]


#### Cluster 5

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1,2] + list(range(5, toronto_merged.shape[1]))]]
